<a href="https://colab.research.google.com/github/VladislavTokarev02/technical/blob/main/test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тестовое задание для кандидата в команду валидации Альфа-Банка

**Привет!** На связи команда валидации Альфа-Банка. Мы отвечаем за то, чтобы модели, которые используются для принятия решений, были надежными, справедливыми и понятными. Твое задание — провести моделирование и валидацию кредитной модели. Удачи, и помни: мы верим в твои силы! 💪  

---

## Цель  
Проверить навыки анализа данных, валидации ML-моделей и умение формулировать выводы для бизнеса.  

---

## Датасет  
**Название:** [Give Me Some Credit](https://www.kaggle.com/c/GiveMeSomeCredit/data)  
**Описание:**  
Прогнозирование дефолта заемщика на основе финансовых и демографических признаков.  

**Признаки:**  
- `RevolvingUtilizationOfUnsecuredLines` (использование кредитных линий),  
- `age`, `MonthlyIncome`, `NumberOfDependents` и др.  

**Целевая переменная:** `SeriousDlqin2yrs` (дефолт: 0/1).  

---

## Задача  
Проведите валидацию модели кредитного скоринга (логистическая регрессия или градиентный бустинг) и подготовьте отчет.  

---

## Шаги  

### 1. Анализ и предобработка данных  
- Обработайте пропуски (например, в `MonthlyIncome`).  
- Исследуйте выбросы (например, возраст < 18 лет).  
- Визуализируйте распределения ключевых признаков.  
- Предложите методы борьбы с дисбалансом классов.  

### 2. Построение и оценка модели  
- Разделите данные на train/validation/test.  
- Обучите модель (логистическая регрессия или CatBoost/XGBoost).  
- Рассчитайте метрики: **AUC-ROC, Precision, Recall, F1-Score**.  
- Проверьте устойчивость модели через кросс-валидацию (5 folds).  

### 3. Интерпретация и этика  
- Выделите **топ-5 признаков**, влияющих на прогноз (SHAP/LIME).  
- Проверьте логичность влияния признаков (например, высокая долговая нагрузка → выше риск дефолта).  
- Оцените fairness модели: сравните метрики (FPR, TPR) для групп (например, **молодые** vs **старше 40 лет**).  

### 4. Отчет  
Подготовьте общие выводы по проведенной валидации, включив:  
- Выводы о качестве модели и её ограничениях.  
- Рекомендации по улучшению (например, сбор дополнительных данных).  
- Пример: *«Как изменится прогноз, если у заемщика появится иждивенец?»*  

---

## Технические требования  
- Язык: **Python** (Jupyter Notebook).  
- Код должен быть читаемым и содержать комментарии.  

---

## Критерии оценки  
1. Глубина анализа данных и обработки выбросов.  
2. Корректность выбранных метрик и их интерпретация.  
3. Качество визуализаций (распределения, важность признаков).  
4. Практичность рекомендаций в отчете.    

---

**Срок выполнения:** 7 дней.  
**Формат сдачи:**  
- Ноутбук в Collab/GitHub-репозиторий.    

---

Это задание покажет, как вы подходите к анализу реальных данных и делаете ML-модели прозрачными для бизнеса. Ждем твою работу! 🚀  

In [3]:
!pip install ydata_profiling -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.1/392.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 71.3 MB/s eta 0:00:00


In [38]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split
from ydata_profiling import ProfileReport
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor

random.seed(42)


pd.set_option('display.max_columns', None)

In [5]:
data_dict = pd.read_excel('Data Dictionary.xls')

Интерпретация столбцов датасета из data_dict

| Имя переменной                      | Описание                                                                                                                                         |
|-----------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| SeriousDlqin2yrs                    | Лицо имело задолженность 90+ дней или более за последние 2 года.                                                                                      |
| RevolvingUtilizationOfUnsecuredLines| Общий баланс по кредитным картам и персональным кредитным линиям (без недвижимости и займов на покупку авто) делённый на сумму кредитных лимитов.     |
| age                                 | Возраст заёмщика в годах.                                                                                                                            |
| NumberOfTime30-59DaysPastDueNotWorse| Количество случаев, когда заёмщик был в просрочке 30–59 дней, но не более, за последние 2 года.                                                      |
| DebtRatio                           | Соотношение ежемесячных платежей по долгам (алименты, расходы на жизнь и т. д.) к ежемесячному валовому доходу.                                       |
| MonthlyIncome                       | Ежемесячный доход.                                                                                                                                   |
| NumberOfOpenCreditLinesAndLoans     | Количество открытых кредитов (автокредит или ипотека) и кредитных линий (например, кредитные карты).                                                 |
| NumberOfTimes90DaysLate             | Количество случаев, когда заёмщик был в просрочке на 90 дней или более.                                                                              |
| NumberRealEstateLoansOrLines        | Количество займов под недвижимость, включая кредитные линии на недвижимость.                                                                         |
| NumberOfTime60-89DaysPastDueNotWorse| Количество случаев, когда заёмщик был в просрочке 60–89 дней, но не более, за последние 2 года.                                                      |
| NumberOfDependents                  | Количество иждивенцев в семье (исключая самого заёмщика, супругу/супруга и т. д.).                                                                    |

In [26]:
train = pd.read_csv('cs-training.csv', index_col = 0).reset_index(drop = True)
train.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [27]:
test = pd.read_csv('cs-test.csv', index_col = 0).reset_index(drop = True)
test.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


In [28]:
train.shape, test.shape

((150000, 11), (101503, 11))

В качестве первого быстрого ознакомления с данными можно воспользоваться дашбородом в ydata_profiling

In [29]:
profile_report = ProfileReport(train, title='EDA Report')

#profile_report
#вывод дашборда на экран

<b> Рассмотрим более детально датасеты

In [30]:
train.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


In [9]:
train.duplicated().sum(), test.duplicated().sum()
# Полных дубликатов нет

(0, 0)

In [16]:
train.isna().mean().round(4).to_frame().sort_values(by=0, ascending = False).style.format('{:.2%}').background_gradient('coolwarm')

,0
MonthlyIncome,19.82%
NumberOfDependents,2.62%
Unnamed: 0,0.00%
SeriousDlqin2yrs,0.00%
RevolvingUtilizationOfUnsecuredLines,0.00%
age,0.00%
NumberOfTime30-59DaysPastDueNotWorse,0.00%
DebtRatio,0.00%
NumberOfOpenCreditLinesAndLoans,0.00%
NumberOfTimes90DaysLate,0.00%


In [17]:
test.isna().mean().round(4).to_frame().sort_values(by=0, ascending = False).style.format('{:.2%}').background_gradient('coolwarm')

,0
SeriousDlqin2yrs,100.00%
MonthlyIncome,19.81%
NumberOfDependents,2.59%
Unnamed: 0,0.00%
RevolvingUtilizationOfUnsecuredLines,0.00%
age,0.00%
NumberOfTime30-59DaysPastDueNotWorse,0.00%
DebtRatio,0.00%
NumberOfOpenCreditLinesAndLoans,0.00%
NumberOfTimes90DaysLate,0.00%


Как мы видим - пропуски есть только в 2 столбцах - ежемесячный доход и количество иждивенцев в семье.

- Количество иждивенцев в семье: доля пропусков немного выше 2.5% - это не столь существенно для анализа, таким образом, можно просто удалить эти пропуски без заполнения. Можно было бы также заполнить модой или выбрать более интересную стратегию, но при таком малом количестве пропусков можно перейти в сторону "чистоты" данных.
- Ежемесячный доход: в простейшем случае можно заполнить медианой, хотя существуют вариации заполнения с учётом k-ближайших соседей, линейной регрессии, k-means. В виду того, что пропусков много (~20% от всех данных) воспользуемся продвинутыми методами.

In [32]:
train.dropna(subset=['NumberOfDependents'], inplace=True)
test.dropna(subset=['NumberOfDependents'], inplace=True)

Первая стратегия: заполнение пропусков MonthlyIncome при помощи KNN

In [ ]:
imputer = KNNImputer(n_neighbors=5)

train_knn = imputer.fit_transform(train)
train_knn = pd.DataFrame(train_knn, columns=train.columns)

test_knn = imputer.transform(test_df)
test_knn = pd.DataFrame(test_knn, columns=test.columns)

Вторая стратегия: заполнение пропусков MonthlyIncome при помощи случайного леса.

In [35]:
train_nan = train[train['MonthlyIncome'].isnull()]
train_no_nan = train.dropna(subset=['MonthlyIncome'])


X_train = train_no_nan.drop(columns=['MonthlyIncome'])
y_train = train_no_nan['MonthlyIncome']


rf = RandomForestRegressor(n_estimators=500, random_state=42)
rf.fit(X_train, y_train)


X_nan = train_nan.drop(columns=['MonthlyIncome'])
predicted_ages = rf_regressor.predict(X_nan)


train_rf.loc[train['MonthlyIncome'].isnull(), 'MonthlyIncome'] = predicted_ages


X_test = test.drop(columns=['MonthlyIncome'])
predicted_ages_test = rf_regressor.predict(X_test)

test_rf['MonthlyIncome'] = predicted_ages_test

NameError: name 'RandomForestRegressor' is not defined